# 인공신경망

<h4>from sklearn.nerual_network import MLPClassifier</h4>

---

## Part1. 분류(Classification)

### 1. 분석 데이터 준비

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [2]:
data1 = pd.read_csv('../breast-cancer-wisconsin.csv')
data1.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [5]:
# 특성 데이터 분류
X = data1[data1.columns[1:-1]]
y = data1[['Class']]

In [6]:
# 훈련, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y, random_state=42)

In [8]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2. 기본모델 적용 (인공신경망)
- MLPClassifier

In [9]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.974609375

In [10]:
# 오차행렬
from sklearn.metrics import confusion_matrix
confusion_train=confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬:\n", confusion_train)

훈련데이터 오차행렬:
 [[328   5]
 [  8 171]]


In [11]:
# 분류예측 모델
from sklearn.metrics import classification_report
cfreport_train=classification_report(y_train, pred_train)
print("분류예측 레포트:\n", cfreport_train)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       333
           1       0.97      0.96      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [12]:
# 테스트 데이터
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9590643274853801

In [13]:
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  2  58]]


In [14]:
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



### 3. Grid Search

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
param = {'hidden_layer_sizes' : [10,30,50,100], 'solver' : ['sgd','adam'],
         'activation' : ['tanh','relu']}
grid_search = GridSearchCV(MLPClassifier(), param, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'],
                         'hidden_layer_sizes': [10, 30, 50, 100],
                         'solver': ['sgd', 'adam']})

In [18]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'activation': 'relu', 'hidden_layer_sizes': 100, 'solver': 'adam'}
Best Score: 0.9746
TestSet Score: 0.9591


### 4. Random Search

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [20]:
param = {'hidden_layer_sizes' : randint(low=10, high=100), 'solver':['sgd','adam'],
         'activation' : ['tanh','relu']}

In [21]:
random_search = RandomizedSearchCV(MLPClassifier(), param, n_iter=10, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=MLPClassifier(),
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'hidden_layer_sizes': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000277EBDA1340>,
                                        'solver': ['sgd', 'adam']})

In [22]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'activation': 'relu', 'hidden_layer_sizes': 93, 'solver': 'adam'}
Best Score: 0.9746
TestSet Score: 0.9591


---

## Part2. 회귀(Regression)

### 1. 분석 데이터 준비

In [23]:
data2 = pd.read_csv('../house_price.csv')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [24]:
# 특성 데이터 나누기
X=data2[data2.columns[1:5]]
y=data2[["house_value"]]

In [25]:
# 학습, 테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [27]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2. 기본모델 적용
- MLPRegressor

In [31]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor()
model.fit(X_scaled_train, y_train)

# 학습 데이터
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

-2.6791766641965635

In [32]:
# 테스트 데이터
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

-2.6375018644224797

In [35]:
# RMSE 
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)

print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 183073.8711556715
테스트 데이터 RMSE: 182332.49978223993


### 3. 튜닝 모델

In [36]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(64,64,64), activation='relu', random_state=1, max_iter=2000)
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.566197903746314

In [37]:
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.584086684313508

In [38]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 62863.255358058195
테스트 데이터 RMSE: 61654.37310884089
